In [1]:
# import os

# from google.colab import drive

# drive.mount('/content/gdrive', force_remount=True)

# %cd "/content/gdrive/My Drive/Colab Notebooks"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


!pip install -r requirement.txt


In [1]:
import os
import torch
import argparse
import itertools
import matplotlib.pyplot as plt
from typing import Optional

from FinanceSigWGANBase.evaluate import evaluate_generator
from lib.networks import get_generator
from lib.utils import to_numpy, load_obj
from lib.augmentations import parse_augmentations
from lib.test_metrics import get_standard_test_metrics
from lib.trainers.sig_wgan import compute_expected_signature

from FinanceSigWGANBase.utils.trainer import SigWGANTrainer
from FinanceSigWGANBase.utils.plot import plot_signature, plot_test_metrics
from FinanceSigWGANBase.utils.datasets import get_dataset, train_test_split
from FinanceSigWGANBase.utils.utils import set_seed, save_obj, get_sigwgan_experiment_dir, get_config_path, \
    plot_individual_data

os.environ['PYTHONHASHSEED'] = "0"


In [2]:
%cd "./FinanceSigWGANBase"

d:\Python\FinanceSigWGAN\FinanceSigWGANBase


In [4]:
def main(
        data_config: dict,
        dataset: str,
        experiment_dir: str,
        gan_algo: str,
        gan_config: dict,
        generator_config: dict,
        device: str = 'cpu',
        seed: Optional[int] = 0
):
    """
    Full training procedure.
    Includes: initialising the dataset / generator / GAN and training the GAN.
    """

    n_lags = data_config["n_lags"]

    # Get / prepare dataset
    x_real_rolled = get_dataset(dataset, data_config)
    x_real_rolled = x_real_rolled.to(device)
    set_seed(seed)
    print('Total data: ', list(x_real_rolled.shape))
    # exit()
    x_real_train, x_real_test = train_test_split(x_real_rolled, train_test_ratio=0.8)
    x_real_dim: int = x_real_rolled.shape[2]

    # Compute test metrics for train and test set
    test_metrics_train = get_standard_test_metrics(x_real_train)
    test_metrics_test = get_standard_test_metrics(x_real_test)

    # Get generator
    set_seed(seed)
    generator_config.update(output_dim=x_real_dim)
    g = get_generator(**generator_config).to(device)

    # Get GAN
    if gan_algo == 'SigWGAN':
        trainer = SigWGANTrainer(generator=g,
                                 x_real_rolled=x_real_rolled,
                                 test_metrics_train=test_metrics_train,
                                 test_metrics_test=test_metrics_test,
                                 **gan_config
                                 )
    else:
        raise NotImplementedError()

    # Start training
    set_seed(seed)
    trainer.fit(device=device)

    # Store relevant training results
    # save_obj(to_numpy(x_real_rolled), os.path.join(experiment_dir, 'x_real_rolled.pkl'))
    # save_obj(to_numpy(x_real_train), os.path.join(experiment_dir, 'x_real_train.pkl'))
    save_obj(to_numpy(x_real_test), os.path.join(experiment_dir, 'x_real_test.pkl'))
    save_obj(trainer.losses_history, os.path.join(experiment_dir, 'losses_history.pkl'))  # dev of losses / metrics
    save_obj(trainer.G.state_dict(), os.path.join(experiment_dir, 'generator_state_dict.pt'))
    save_obj(trainer.G, os.path.join(experiment_dir, 'generator_full_model.pt'))
    save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.pkl'))

    loss_history = os.path.join(experiment_dir, 'LossHistory')
    os.makedirs(loss_history, exist_ok=True)

    plt.plot(trainer.losses_history['sig_w1_loss'], alpha=0.8)
    plt.grid()
    plt.yscale('log')
    plt.savefig(os.path.join(loss_history, 'sig_loss.png'))
    plt.close()

    plot_test_metrics(trainer.test_metrics_train, trainer.losses_history, 'train', locate_dir=loss_history)

    plot_test_metrics(trainer.test_metrics_train, trainer.losses_history, 'test', locate_dir=loss_history)

    with torch.no_grad():
        x_fake = g(1024, n_lags, device)

    for i in range(x_real_dim):
        plt.plot(to_numpy(x_fake[:400, :, i]).T, 'C%s' % i, alpha=0.1)
    plt.savefig(os.path.join(experiment_dir, 'x_fake.png'))
    plt.close()

    for i in range(x_real_dim):
        random_indices = torch.randint(0, x_real_rolled.shape[0], (400,))
        plt.plot(to_numpy(x_real_rolled[random_indices, :, i]).T, 'C%s' % i, alpha=0.1)
    plt.savefig(os.path.join(experiment_dir, 'x_real.png'))
    plt.close()

    plot_individual_data(dataset=data_config['name'], locate_dir=os.path.join(experiment_dir, 'RealDataFigure'))

    evaluate_generator(experiment_dir, batch_size=5000, )

    plot_signature(trainer.sig_w1_metric.expected_signature_mu)
    plt.savefig(os.path.join(experiment_dir, 'sig_real.png'))
    plt.close()

    plot_signature(trainer.sig_w1_metric.expected_signature_mu)
    plot_signature(compute_expected_signature(x_fake,
                                              trainer.sig_w1_metric.depth, trainer.sig_w1_metric.augmentations))
    plt.savefig(os.path.join(experiment_dir, 'sig_real_fake.png'))
    plt.close()

In [5]:
def benchmark_sigwgan(
        datasets=('BINANCE', 'STABLECOIN'),
        generators=('LogSigRNN', 'LSTM'),
        n_seeds={"start": 0,"end": 2,"step": 1},
        device='cuda:0',
):
    """ Benchmark for SigWGAN. """
    seeds = list(range(n_seeds["start"],n_seeds["end"],n_seeds["step"]))

    grid = itertools.product(datasets, generators, seeds)

    for dataset, generator, seed in grid:
        print(f"data:{dataset}, gen:{generator}, seed:{seed}")
        data_config = load_obj(get_config_path('', dataset))
        gan_config = load_obj(get_config_path('Trainer', 'trainer_SigWGAN'))
        generator_config = load_obj(get_config_path('Generator', 'gen_' + generator))

        experiment_dir = get_sigwgan_experiment_dir(dataset, generator, 'SigWGAN', seed)

        if not os.path.exists(experiment_dir):
            os.makedirs(experiment_dir)

        save_obj(data_config, os.path.join(experiment_dir, 'data_config.json'))
        save_obj(gan_config, os.path.join(experiment_dir, 'gen_config.json'))
        save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.json'))

        if gan_config.get('augmentations') is not None:
            gan_config['augmentations'] = parse_augmentations(gan_config.get('augmentations'))

        if generator_config.get('augmentations') is not None:
            generator_config['augmentations'] = parse_augmentations(generator_config.get('augmentations'))

        if generator_config['generator_type'] == 'LogSigRNN':
            generator_config['n_lags'] = data_config['n_lags']

        save_obj(data_config, os.path.join(experiment_dir, 'data_config.pkl'))
        save_obj(gan_config, os.path.join(experiment_dir, 'gen_config.pkl'))
        save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.pkl'))

        print('Training: %s' % experiment_dir.split('/')[-2:])

        main(
            dataset=dataset,
            data_config=data_config,
            device=device,
            experiment_dir=experiment_dir,
            gan_algo='SigWGAN',
            seed=seed,
            gan_config=gan_config,
            generator_config=generator_config,
        )

In [7]:
if torch.cuda.is_available():
    compute_device = 'cuda:0'
else:
    compute_device = 'cpu'

# target_dataset = os.listdir('./datasets')
# target_dataset.remove('Uniswap')
# target_dataset.append('Uniswap')
# target_dataset = ('BINANCE',)
# target_dataset = ('WrappedBitcoin',)
# target_dataset = ('STABLECOIN',)
# target_dataset = ('Uniswap',)
# target_dataset = ('BINANCE', 'STABLECOIN')

n_seeds = {
    "start": 0,
    "end": 100,
    "step": 5
}

target_dataset = ('MyBinance', )
training_generators = ('LogSigRNN',)
# training_generators = ('LSTM',)
# training_generators = ('LogSigRNN', 'LSTM')
benchmark_sigwgan(datasets=target_dataset, 
                  generators=training_generators,
                  n_seeds=n_seeds,
                  device=compute_device)


data:MyBinance, gen:LogSigRNN, seed:0
Training: ['MyBinance', 'SigWGAN_LogSigRNN_0']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [22470, 24, 3]
	Example : tensor([[0.0000, 0.0000, 0.0000],
        [0.0082, 0.0018, 0.0029],
        [0.0109, 0.0040, 0.0149]])
Total data:  [22470, 24, 3]


sig-w1 loss: 7.211739e-01: 100%|██████████| 250/250 [00:51<00:00,  4.89it/s]


data:MyBinance, gen:LogSigRNN, seed:5
Training: ['MyBinance', 'SigWGAN_LogSigRNN_5']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [22470, 24, 3]
	Example : tensor([[0.0000, 0.0000, 0.0000],
        [0.0082, 0.0018, 0.0029],
        [0.0109, 0.0040, 0.0149]])
Total data:  [22470, 24, 3]


sig-w1 loss: 6.618428e-01: 100%|██████████| 250/250 [00:53<00:00,  4.69it/s]


data:MyBinance, gen:LogSigRNN, seed:10
Training: ['MyBinance', 'SigWGAN_LogSigRNN_10']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [22470, 24, 3]
	Example : tensor([[0.0000, 0.0000, 0.0000],
        [0.0082, 0.0018, 0.0029],
        [0.0109, 0.0040, 0.0149]])
Total data:  [22470, 24, 3]


...

data:MyBinance, gen:LogSigRNN, seed:45
Training: ['MyBinance', 'SigWGAN_LogSigRNN_45']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [22470, 24, 3]
	Example : tensor([[0.0000, 0.0000, 0.0000],
        [0.0082, 0.0018, 0.0029],
        [0.0109, 0.0040, 0.0149]])
Total data:  [22470, 24, 3]


sig-w1 loss: 1.032302e+01: 100%|██████████| 250/250 [00:53<00:00,  4.68it/s]


data:MyBinance, gen:LogSigRNN, seed:50
Training: ['MyBinance', 'SigWGAN_LogSigRNN_50']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [22470, 24, 3]
	Example : tensor([[0.0000, 0.0000, 0.0000],
        [0.0082, 0.0018, 0.0029],
        [0.0109, 0.0040, 0.0149]])
Total data:  [22470, 24, 3]


sig-w1 loss: 4.320805e+01:   3%|▎         | 7/250 [00:01<01:09,  3.51it/s]


KeyboardInterrupt: 